# Spatial search
---

## Introduction
---
The goal here is to create a simple way to overlay leaflet maps with GeoPandas dataframes and plot the aggregate numeric data derived from them.
The most interesting part is the implementation of a "graphical query"
whereby a user can select and aggregate GeoDataframes rows by simply drawing selection polygons on the underlaying leaflet map.

## Implementation
---

In [1]:
# geometry
from shapely.geometry import mapping, shape

# I/O
import geopandas as gpd

# widgets
import bqplot as bq
import ipyleaflet as ipyl
import ipywidgets as ipyw

# numerics
import numpy as np

In [2]:
df = gpd.read_file('../data/country_centroids')
df.head()

,affil,cartodb_id,dms_lat,dms_long,dsg,fips10,full_name,geometry,iso3136,jog,lat,long,mgrs,mod_date,short_name
0,None,4,330000.0,660000.0,PCLI,AF,Islamic Republic of Afghanistan,POINT (66 33),AF,NI42-09,33.000000,66.0,42STB1970055286,4/10/2009,Afghanistan
1,None,7,410000.0,200000.0,PCLI,AL,Republic of Albania,POINT (20 41),AL,NK34-08,41.000000,20.0,34TDL1589839239,2/28/2007,Albania
2,None,5,280000.0,30000.0,PCLI,AG,People's Democratic Republic of Algeria,POINT (3 28),DZ,NH31-15,28.000000,3.0,31REL0000097202,3/3/2011,Algeria
3,US,12,-142000.0,-1700000.0,PCLD,AS,Territory of American Samoa,POINT (-170 -14.333333),AS,None,-14.333333,-170.0,1802701,10/6/1998,American Samoa
4,None,9,423000.0,13000.0,PCLI,AN,Principality of Andorra,POINT (1.5 42.5),AD,NK31-04,42.500000,1.5,31TCH7675006383,2/28/2007,Andorra


We generate mock data for testing purposes. In practice, these indicators can be added from another dataframe, from an API/sensor data stream and even directly from an HTML web page by using modules like beautifulsoup4 

In [3]:
# mock data
df['val1'], df['val2'], df['val3'], df['val4'] = tuple(np.random.rand(4, 258) * 1e6) 
df.head()

,affil,cartodb_id,dms_lat,dms_long,dsg,fips10,full_name,geometry,iso3136,jog,lat,long,mgrs,mod_date,short_name,val1,val2,val3,val4
0,None,4,330000.0,660000.0,PCLI,AF,Islamic Republic of Afghanistan,POINT (66 33),AF,NI42-09,33.000000,66.0,42STB1970055286,4/10/2009,Afghanistan,616524.043821,423676.803494,99713.295154,307842.495760
1,None,7,410000.0,200000.0,PCLI,AL,Republic of Albania,POINT (20 41),AL,NK34-08,41.000000,20.0,34TDL1589839239,2/28/2007,Albania,291600.234767,280095.475228,261467.594751,724900.307263
2,None,5,280000.0,30000.0,PCLI,AG,People's Democratic Republic of Algeria,POINT (3 28),DZ,NH31-15,28.000000,3.0,31REL0000097202,3/3/2011,Algeria,101079.562601,721500.013416,232311.611477,104148.505343
3,US,12,-142000.0,-1700000.0,PCLD,AS,Territory of American Samoa,POINT (-170 -14.333333),AS,None,-14.333333,-170.0,1802701,10/6/1998,American Samoa,432193.075921,991378.904293,164404.905366,830858.415492
4,None,9,423000.0,13000.0,PCLI,AN,Principality of Andorra,POINT (1.5 42.5),AD,NK31-04,42.500000,1.5,31TCH7675006383,2/28/2007,Andorra,815191.655088,414812.593530,798943.665265,860763.692796


We instantiate an ipyleaflet map a drawing control to serve as visual support for the "geographical search" functionality

In [4]:
m= ipyl.Map(scroll_wheel_zoom=True, center=[-0.3515602939922709, 22.5], 
            zoom=1, layout=ipyw.Layout(width='45%', height='450px'))

dc = ipyl.DrawControl(polygon={'shapeOptions': {'color': 'green', 'weight': 2, 'clickable': True}})
m.add_control(dc)

for centroid in df.geometry:
    m.add_layer(ipyl.GeoJSON(data=mapping(centroid)))

In [5]:
# while len(m.layers) > 1:
#     m.remove_layer(m.layers[-1])

We develop the spatial search functionality by using shapely and plot the aggregate data with bqplot. For the next block to have a functional output, you need a live kernel. Download the notebook and give it a try!

In [6]:
ordinal_features_scale = bq.OrdinalScale()
values_features_scale = bq.LinearScale()
ord_axis = bq.Axis(scale=ordinal_features_scale)
value_axis = bq.Axis(scale=values_features_scale, orientation='vertical')

indicators = df.loc[:, df.dtypes == np.float64].columns.values

bars = bq.Bars(x=indicators, y=np.ones(len(indicators)), 
               scales={
               'x': ordinal_features_scale,
               'y': values_features_scale
                      }, base=1.0)

indication = bq.Label(x=[0.25], y=[0.5], text=['Draw a polygon over the map'], font_size='100px', color=['green'])

figure = bq.Figure(axes=[value_axis, ord_axis], marks=[bars, indication],
                   title='Indicators',
                   animation_duration=500, layout=ipyw.Layout(min_width='55%'))

def scores(geo_json):
    # geometric selection
    polygon_data = df[df.geometry.within(shape(geo_json['geometry']))]
    # projection on indicators columns
    polygon_data = polygon_data[indicators]
    if not polygon_data.empty:
        return polygon_data.mean(axis=0)
    else:
        return np.ones(len(indicators))



def handle_draw(self, action, geo_json):
    figure.marks = [bars]
    bars.y = scores(geo_json)
    
    
dc.on_draw(handle_draw)

ipyw.HBox([m, figure])

## What comes next ?
---
Ideally:
1. Make the spatial search more data-agnostic and add other modes of spatial selection
2. Add the possibility to choose a custom aggregation function, aside from those proposed by numpy
3. Try to complete the UI with a qgrid based widget
4. Solve real-world problems with the resulting application and document the process

If you want to help, let us know and don't hesitate to make some pull requests